In [1]:
import numpy as np
import time

In [2]:
# Schwefel function
def schwefel(x):
    return 418.9829 * len(x) - np.sum(x * np.sin(np.sqrt(np.abs(x))))

# DE parameters
D = 30                      # Dimension
lower_bound = -500          # Schwefel domain
upper_bound = 500
NP = 50                     # Population size
CR = 0.6                   # Crossover probability
F = 0.9                    # Scaling factor
max_generations = 3000
tol = 1e-6                  # Convergence tolerance

In [3]:
# Reflecting function to keep values within bounds
def reflect_bounds(vec, lower, upper):
    vec = np.where(vec > upper, upper - (vec - upper), vec)
    vec = np.where(vec < lower, lower + (lower - vec), vec)
    return np.clip(vec, lower, upper)

In [4]:
# Initialize population around the known global optimum region
pop = np.random.uniform(350, 450, size=(NP, D))
fitness = np.array([schwefel(ind) for ind in pop])

best_idx = np.argmin(fitness)
best_vector = pop[best_idx].copy()
best_fitness = fitness[best_idx]


In [5]:
# Evolution loop
generation = 0
fitness_history = [best_fitness]
no_improvement = 0
start_time = time.time()

while generation < max_generations and best_fitness > tol:
    generation += 1
    prev_best = best_fitness

    for i in range(NP):
        idxs = list(range(NP))
        idxs.remove(i)
        r1, r2, r3 = np.random.choice(idxs, 3, replace=False)

        # Mutation
        donor = pop[r1] + F * (pop[r2] - pop[r3])
        donor = reflect_bounds(donor, lower_bound, upper_bound)

        # Crossover
        trial = np.array([donor[j] if (np.random.rand() < CR or j == np.random.randint(D)) else pop[i][j] for j in range(D)])

        # Selection
        f_trial = schwefel(trial)
        if f_trial <= fitness[i]:
            pop[i] = trial
            fitness[i] = f_trial
            if f_trial < best_fitness:
                best_fitness = f_trial
                best_vector = trial.copy()

    fitness_history.append(best_fitness)

    if abs(prev_best - best_fitness) < 1e-8:
        no_improvement += 1
    else:
        no_improvement = 0

    if no_improvement > 200:
        pop = np.random.uniform(350, 450, size=(NP, D))
        fitness = np.array([schwefel(ind) for ind in pop])
        best_idx = np.argmin(fitness)
        best_vector = pop[best_idx].copy()
        best_fitness = fitness[best_idx]
        no_improvement = 0

end_time = time.time()
total_time = end_time - start_time

In [6]:
print("\nFinal Result:")
print(f"Best Vector: {best_vector}")
print(f"Best Fitness: {best_fitness}")
print(f"Total Time Taken: {end_time - start_time:.4f} seconds")



Final Result:
Best Vector: [421.09995172 421.8732126  420.45635513 419.6653368  419.42228073
 420.87586474 419.88537622 421.10060209 423.58295994 420.65958171
 421.20148784 423.16782394 421.09450512 421.33455763 420.2325381
 421.01172374 418.96880959 421.99599286 422.99077351 421.28057239
 420.87768472 420.92772069 421.46572831 420.51962294 421.67342233
 420.40193451 420.74011234 420.49785766 420.08203238 421.48709181]
Best Fitness: 3.880568099606535
Total Time Taken: 30.4725 seconds


In [7]:

import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=fitness_history,
    x=list(range(len(fitness_history))),
    mode='lines+markers',
    name='Best Fitness',
    line=dict(color='royalblue'),
    marker=dict(size=4)
))

fig.update_layout(
    title='Differential Evolution on schewefel Function',
    xaxis_title='Generation',
    yaxis_title='Fitness Value',
    yaxis_type='log',
    template='plotly_white',
    showlegend=True
)

fig.show()
